# MCPStore 功能测试 Notebook

这个 Jupyter Notebook 基于 `同步测试plus.py` 重构，提供交互式的 MCPStore 功能测试环境。

## 📋 测试覆盖范围

- ✅ **基础功能测试**: Store初始化、服务注册、工具列表
- ✅ **服务状态管理**: 健康检查、状态查询、服务重启
- ✅ **高级服务管理**: 配置更新、批量操作、服务详情
- ✅ **Agent模式测试**: Agent隔离、专属服务、上下文切换
- ✅ **配置管理**: 统一配置、重置操作、默认配置恢复
- ✅ **工具执行**: 地图工具、思维工具、参数测试
- ✅ **批量操作**: 批量添加、删除服务
- ✅ **错误处理**: 异常场景、无效配置测试

## 🚀 开始测试

每个单元格都是独立的测试，可以单独运行。按顺序执行或选择特定测试。

## 1. 环境准备

In [ ]:
# 导入必要的库
import time
import json
import sys
from datetime import datetime

# 添加项目路径
sys.path.append('src')
from mcpstore.jupyter_helper import setup_mcpstore, add_service, list_tools, use_tool
from mcpstore import MCPStore

print("🚀 MCPStore 测试环境已准备就绪!")
print(f"📅 测试时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("📝 每个单元格都是独立的测试，可以单独运行")

## 2. 初始化 MCPStore

In [2]:
# 初始化 MCPStore
print("🔍 初始化 MCPStore")
print("="*50)


store = MCPStore.setup_store()
print("✅ Store 初始化成功")
print(f"📦 Store 类型: {type(store).__name__}")


🔍 初始化 MCPStore
✅ Store 初始化成功
📦 Store 类型: MCPStore


## 3. 查看当前配置

In [3]:
# 查看当前 MCP 配置
print("🔍 查看当前 MCP 配置")
print("="*50)

config = store.for_store().show_mcpconfig()
server_count = len(config.get('mcpServers', {}))
print(f"✅ 获取配置成功，找到 {server_count} 个配置项")

# 显示配置详情
if server_count > 0:
    print("\n📋 配置的服务:")
    for i, (name, conf) in enumerate(config.get('mcpServers', {}).items()):
        if i >= 5:  # 只显示前5个
            print(f"   ... 还有 {server_count - 5} 个服务")
            break
        service_type = "URL" if conf.get('url') else "Command"
        print(f"   • {name} ({service_type})")
        if conf.get('url'):
            print(f"     URL: {conf['url'][:50]}..." if len(conf['url']) > 50 else f"     URL: {conf['url']}")
        elif conf.get('command'):
            print(f"     Command: {conf['command']}")

🔍 查看当前 MCP 配置
✅ 获取配置成功，找到 3 个配置项

📋 配置的服务:
   • mcpstore-demo-weather (URL)
     URL: http://59.110.160.18:21923/mcp
   • agent_exclusive_service (URL)
     URL: http://59.110.160.18:21923/mcp
   • analysis_service (URL)
     URL: http://59.110.160.18:21923/mcp


## 4. 注册服务

In [4]:
# 注册配置文件中的所有服务
print("🔍 注册配置文件中的服务")
print("="*50)
services_before = store.for_store().list_services()
print(f"📊 注册前服务数量: {len(services_before)}")

🔍 注册配置文件中的服务
📊 注册前服务数量: 0


In [5]:

start_time = time.time()
store.for_store().add_service()  # 注册所有配置的服务
elapsed = time.time() - start_time
print(f"✅ 服务注册成功，耗时 {elapsed:.3f}s")

[INFO][add_service] 当前模式: STORE, agent_id: main_client
[INFO][add_service] STORE模式-全量注册所有服务
[INFO][register_json_service] 默认全量注册
[DEBUG][add_service] 首次注册服务 - agent_id=client_20250629090903_mo2g5n, name=mcpstore-demo-weather
[INFO][register_json_service] 成功注册服务: mcpstore-demo-weather
[DEBUG][add_service] 首次注册服务 - agent_id=client_20250629090904_v4ggzq, name=agent_exclusive_service
[INFO][register_json_service] 成功注册服务: agent_exclusive_service
[DEBUG][add_service] 首次注册服务 - agent_id=client_20250629090906_p54gfr, name=analysis_service
[INFO][register_json_service] 成功注册服务: analysis_service
[INFO][add_service] 注册结果: success=True message=None client_id='main_client' service_names=['mcpstore-demo-weather', 'agent_exclusive_service', 'analysis_service'] config={'client_ids': ['client_20250629090903_mo2g5n', 'client_20250629090904_v4ggzq', 'client_20250629090906_p54gfr'], 'services': ['mcpstore-demo-weather', 'agent_exclusive_service', 'analysis_service']}
✅ 服务注册成功，耗时 5.523s


In [6]:

# 注册后的服务数量
services_after = store.for_store().list_services()
print(f"📊 注册后服务数量: {len(services_after)}")
print(f"📈 新增服务数量: {len(services_after) - len(services_before)}")

# 显示新注册的服务
if len(services_after) > len(services_before):
    print("\n🆕 新注册的服务:")
    before_names = {s.name for s in services_before}
    for service in services_after:
        if service.name not in before_names:
            print(f"   • {service.name} ({service.status})")


📊 注册后服务数量: 3
📈 新增服务数量: 3

🆕 新注册的服务:
   • mcpstore-demo-weather (healthy)
   • agent_exclusive_service (healthy)
   • analysis_service (healthy)


## 5. 获取服务列表

In [7]:
# 获取当前所有服务列表
print("🔍 获取服务列表")
print("="*50)


start_time = time.time()
services = store.for_store().list_services()
elapsed = time.time() - start_time

print(f"✅ 获取服务列表成功，找到 {len(services)} 个服务")
print(f"⏱️ 耗时: {elapsed:.3f}s")

🔍 获取服务列表
✅ 获取服务列表成功，找到 3 个服务
⏱️ 耗时: 4.221s


In [8]:

print("\n📋 服务详情:")
healthy_count = 0
for i, service in enumerate(services):
    if i >= 10:  # 只显示前10个
        print(f"   ... 还有 {len(services) - 10} 个服务")
        break
    
    status_icon = "✅" if service.status == "healthy" else "❌"
    if service.status == "healthy":
        healthy_count += 1
    
    print(f"   {status_icon} {service.name}")
    print(f"      状态: {service.status}")
    print(f"      工具数: {service.tool_count}")
    print(f"      传输类型: {service.transport_type}")
    if hasattr(service, 'url') and service.url:
        url_display = service.url[:50] + "..." if len(service.url) > 50 else service.url
        print(f"      URL: {url_display}")
    print(f"📊 健康服务: {healthy_count}/{len(services)}")


📋 服务详情:
   ✅ mcpstore-demo-weather
      状态: healthy
      工具数: 4
      传输类型: TransportType.STREAMABLE_HTTP
      URL: http://59.110.160.18:21923/mcp
📊 健康服务: 1/3
   ✅ agent_exclusive_service
      状态: healthy
      工具数: 4
      传输类型: TransportType.STREAMABLE_HTTP
      URL: http://59.110.160.18:21923/mcp
📊 健康服务: 2/3
   ✅ analysis_service
      状态: healthy
      工具数: 4
      传输类型: TransportType.STREAMABLE_HTTP
      URL: http://59.110.160.18:21923/mcp
📊 健康服务: 3/3


## 6. 获取工具列表

In [9]:
# 获取当前所有工具列表
print("🔍 获取工具列表")
print("="*50)


start_time = time.time()
tools = store.for_store().list_tools()
elapsed = time.time() - start_time

print(f"✅ 获取工具列表成功，找到 {len(tools)} 个工具")
print(f"⏱️ 耗时: {elapsed:.3f}s")
        

🔍 获取工具列表
✅ 获取工具列表成功，找到 12 个工具
⏱️ 耗时: 0.001s


In [10]:
if tools:
    # 按服务分类工具
    tool_services = {}
    for tool in tools:
        service = tool.service_name
        if service not in tool_services:
            tool_services[service] = []
        tool_services[service].append(tool)
    
    print(f"\n📊 工具分布 (共 {len(tool_services)} 个服务):")
    for service, service_tools in list(tool_services.items()): 
        print(f"\n🔧 {service} ({len(service_tools)} 个工具):")
        for i, tool in enumerate(service_tools):  
            print(f"   • {tool.name}")
            if tool.description:
                desc = tool.description[:60] + "..." if len(tool.description) > 60 else tool.description
                print(f"     描述: {desc}")
else:
    print("⚠️ 没有找到任何工具")
    


📊 工具分布 (共 3 个服务):

🔧 mcpstore-demo-weather (4 个工具):
   • mcpstore-demo-weather_get_current_weather
     描述: 获取指定城市的当前天气信息，包括温度和天气状况
   • mcpstore-demo-weather_get_weather_forecast
     描述: 获取指定城市未来几天的天气预报，默认3天
   • mcpstore-demo-weather_get_air_quality
     描述: 获取指定城市的空气质量指数(AQI)信息
   • mcpstore-demo-weather_search_weather
     描述: 搜索天气相关信息

🔧 agent_exclusive_service (4 个工具):
   • agent_exclusive_service_get_current_weather
     描述: 获取指定城市的当前天气信息，包括温度和天气状况
   • agent_exclusive_service_get_weather_forecast
     描述: 获取指定城市未来几天的天气预报，默认3天
   • agent_exclusive_service_get_air_quality
     描述: 获取指定城市的空气质量指数(AQI)信息
   • agent_exclusive_service_search_weather
     描述: 搜索天气相关信息

🔧 analysis_service (4 个工具):
   • analysis_service_get_current_weather
     描述: 获取指定城市的当前天气信息，包括温度和天气状况
   • analysis_service_get_weather_forecast
     描述: 获取指定城市未来几天的天气预报，默认3天
   • analysis_service_get_air_quality
     描述: 获取指定城市的空气质量指数(AQI)信息
   • analysis_service_search_weather
     描述: 搜索天气相关信息


## 7. 服务健康检查

In [11]:
# 执行服务健康检查
print("🔍 服务健康检查")
print("="*50)

start_time = time.time()
health_status = store.for_store().check_services()
elapsed = time.time() - start_time
print(health_status)

🔍 服务健康检查
{'orchestrator_status': 'running', 'active_services': 3, 'services': [{'name': 'mcpstore-demo-weather', 'url': 'http://59.110.160.18:21923/mcp', 'transport_type': 'streamable-http', 'status': 'healthy', 'command': None, 'args': None, 'package_name': None}, {'name': 'agent_exclusive_service', 'url': 'http://59.110.160.18:21923/mcp', 'transport_type': '', 'status': 'healthy', 'command': None, 'args': None, 'package_name': None}, {'name': 'analysis_service', 'url': 'http://59.110.160.18:21923/mcp', 'transport_type': '', 'status': 'healthy', 'command': None, 'args': None, 'package_name': None}]}


In [12]:

# 从新的数据结构中获取服务列表
services_list = health_status.get('services', [])

healthy_count = len([s for s in services_list if s.get('status') == 'healthy'])
total_count = len(services_list)
orchestrator_status = health_status.get('orchestrator_status', 'unknown')

print(f"✅ 健康检查完成 (总控制器状态: {orchestrator_status})")
print(f"📊 {healthy_count}/{total_count} 服务健康")
print(f"⏱️ 耗时: {elapsed:.3f}s")

if services_list:
    print("\n🏥 健康状态详情:")
    for service in services_list:
        # 从服务字典中获取信息
        name = service.get('name', 'Unknown')
        health = service.get('status', 'unknown')
        transport = service.get('transport_type', 'N/A')
        
        # 根据健康状态选择图标
        if health == 'healthy':
            icon = "✅"
        elif health == 'unhealthy':
            icon = "❌"
        else:
            icon = "⚠️"
        
        # 打印基本信息
        print(f"   {icon} {name}")
        print(f"      状态: {health}")
        print(f"      传输类型: {transport}")
        
        # 根据传输类型打印连接详情
        if transport == 'stdio':
            command = service.get('command', '')
            args = ' '.join(service.get('args', []))
            print(f"      命令: {command} {args}")
        elif service.get('url'):
            url = service.get('url')
            print(f"      URL: {url}")
        
        print() # 打印一个空行以分隔条目
else:
    print("⚠️ 没有找到任何服务进行健康检查")

✅ 健康检查完成 (总控制器状态: running)
📊 3/3 服务健康
⏱️ 耗时: 1.075s

🏥 健康状态详情:
   ✅ mcpstore-demo-weather
      状态: healthy
      传输类型: streamable-http
      URL: http://59.110.160.18:21923/mcp

   ✅ agent_exclusive_service
      状态: healthy
      传输类型: 
      URL: http://59.110.160.18:21923/mcp

   ✅ analysis_service
      状态: healthy
      传输类型: 
      URL: http://59.110.160.18:21923/mcp



## 8. 单个服务状态查询

In [13]:
# 查询单个服务的详细状态
print("🔍 单个服务状态查询")
print("="*50)
services = store.for_store().list_services()
if services:
    test_service = services[0]
    service_name = test_service.name
    print(f"🎯 测试服务: {service_name}")
    

🔍 单个服务状态查询
🎯 测试服务: mcpstore-demo-weather


In [14]:

status = store.for_store().get_service_status(service_name)
print("✅ 获取服务状态成功")

print("\n📋 服务状态详情:")
for key, value in status.items():
    print(f"   • {key}: {value}")

# 验证必要字段
expected_fields = ['name', 'status', 'connected', 'tool_count']
print("\n🔍 字段完整性检查:")
for field in expected_fields:
    if field in status:
        print(f"   ✅ {field}: 存在")
    else:
        print(f"   ❌ {field}: 缺失")


[INFO][get_service_info] STORE模式-在main_client中查找服务: mcpstore-demo-weather
✅ 获取服务状态成功

📋 服务状态详情:
   • name: mcpstore-demo-weather
   • status: healthy
   • connected: True
   • tool_count: 4
   • last_heartbeat: 2025-06-29 01:21:43.781482
   • transport_type: TransportType.STREAMABLE_HTTP

🔍 字段完整性检查:
   ✅ name: 存在
   ✅ status: 存在
   ✅ connected: 存在
   ✅ tool_count: 存在


In [15]:

# 测试无效服务名
print("\n🔍 测试无效服务名:")
try:
    invalid_status = store.for_store().get_service_status("nonexistent_service_12345")
    print(f"⚠️ 意外成功: {invalid_status}")
except Exception as e:
    print(f"✅ 预期错误: {type(e).__name__} - {e}")

    



🔍 测试无效服务名:
[INFO][get_service_info] STORE模式-在main_client中查找服务: nonexistent_service_12345
⚠️ 意外成功: {'name': 'nonexistent_service_12345', 'status': 'not_found', 'connected': False, 'tool_count': 0, 'last_heartbeat': None, 'transport_type': None}


## 9. Agent 模式测试 - 创建 Agent

In [16]:
# 测试 Agent 模式 - 创建专属 Agent
print("🔍 Agent 模式测试 - 创建专属 Agent")
print("="*50)

# 定义测试 Agent
test_agent_id = "test_navigation_agent"
print(f"🤖 创建测试 Agent: {test_agent_id}")

agent_service_config = {
    "name": "agent_exclusive_service",
    "url": "http://59.110.160.18:21923/mcp"
}

print(f"\n📦 为 Agent 添加专属服务: {agent_service_config['name']}")
store.for_agent(test_agent_id).add_service(agent_service_config)
print("✅ Agent 专属服务添加成功")


🔍 Agent 模式测试 - 创建专属 Agent
🤖 创建测试 Agent: test_navigation_agent

📦 为 Agent 添加专属服务: agent_exclusive_service
[INFO][add_service] 当前模式: AGENT, agent_id: test_navigation_agent
[INFO][add_service] 成功处理同名服务: agent_exclusive_service
[INFO][add_service] 注册服务到Registry，使用client_ids: ['client_20250628234812_fqltj0']
[DEBUG][add_service] 首次注册服务 - agent_id=client_20250628234812_fqltj0, name=agent_exclusive_service
[INFO][add_service] 成功注册client client_20250628234812_fqltj0 到Registry
[INFO][add_service] 服务配置更新和Registry注册完成
✅ Agent 专属服务添加成功


In [17]:
# 获取 Agent 服务列表
agent_services = store.for_agent(test_agent_id).list_services()
print(f"✅ Agent 服务列表: {len(agent_services)} 个服务")

if agent_services:
    print("\n📋 Agent 专属服务:")
    for service in agent_services:
        print(f"   • {service.name} ({service.status})")

✅ Agent 服务列表: 1 个服务

📋 Agent 专属服务:
   • agent_exclusive_service (healthy)


In [18]:

# 获取 Agent 工具列表
agent_tools = store.for_agent(test_agent_id).list_tools()
print(f"✅ Agent 工具列表: {len(agent_tools)} 个工具")

if agent_tools:
    print("\n🔧 Agent 专属工具:")
    for tool in agent_tools:  # 显示前3个
        print(f"   • {tool.name}")
        if tool.description:
            desc = tool.description[:50] + "..." if len(tool.description) > 50 else tool.description
            print(f"     {desc}")


✅ Agent 工具列表: 4 个工具

🔧 Agent 专属工具:
   • agent_exclusive_service_get_current_weather
     获取指定城市的当前天气信息，包括温度和天气状况
   • agent_exclusive_service_get_weather_forecast
     获取指定城市未来几天的天气预报，默认3天
   • agent_exclusive_service_get_air_quality
     获取指定城市的空气质量指数(AQI)信息
   • agent_exclusive_service_search_weather
     搜索天气相关信息


## 10. Agent 上下文隔离验证

In [19]:
# 验证 Agent 和 Store 之间的上下文隔离
print("🔍 Agent 上下文隔离验证")
print("="*50)

store_services = store.for_store().list_services()
store_service_names = {s.name for s in store_services}

# 获取 Agent 级别的服务
test_agent_id = "test_navigation_agent"
agent_services = store.for_agent(test_agent_id).list_services()
agent_service_names = {s.name for s in agent_services}

print(f"📊 Store 级别服务数量: {len(store_services)}")
print(f"📊 Agent 级别服务数量: {len(agent_services)}")

# 检查服务名称重叠
overlap = store_service_names.intersection(agent_service_names)
agent_exclusive = agent_service_names - store_service_names
store_exclusive = store_service_names - agent_service_names

print(f"\n🔍 隔离性分析:")
print(f"   • 重叠服务: {len(overlap)} 个")
print(f"   • Agent 专属服务: {len(agent_exclusive)} 个")
print(f"   • Store 专属服务: {len(store_exclusive)} 个")

if overlap:
    print(f"\n⚠️ 重叠的服务名称:")
    for name in list(overlap)[:5]:  # 显示前5个
        print(f"   • {name}")

if agent_exclusive:
    print(f"\n✅ Agent 专属服务:")
    for name in agent_exclusive:
        print(f"   • {name}")

# 验证隔离性
if len(agent_exclusive) > 0:
    print("\n✅ 上下文隔离正常：Agent 拥有专属服务")
else:
    print("\n⚠️ 上下文隔离可能有问题：Agent 没有专属服务")


🔍 Agent 上下文隔离验证
📊 Store 级别服务数量: 3
📊 Agent 级别服务数量: 1

🔍 隔离性分析:
   • 重叠服务: 1 个
   • Agent 专属服务: 0 个
   • Store 专属服务: 2 个

⚠️ 重叠的服务名称:
   • agent_exclusive_service

⚠️ 上下文隔离可能有问题：Agent 没有专属服务


In [20]:
# 创建第二个 Agent 进行进一步隔离测试
print(f"\n🤖 创建第二个 Agent 进行隔离测试")
second_agent_id = "test_analysis_agent"

# 为第二个 Agent 添加不同的服务
second_agent_config = {
    "name": "analysis_service",
    "url": "http://59.110.160.18:21923/mcp"
}

store.for_agent(second_agent_id).add_service(second_agent_config)
second_agent_services = store.for_agent(second_agent_id).list_services()
second_agent_names = {s.name for s in second_agent_services}

# 检查两个 Agent 之间的隔离
agent_to_agent_overlap = agent_service_names.intersection(second_agent_names)

print(f"📊 第二个 Agent 服务数量: {len(second_agent_services)}")
print(f"🔍 两个 Agent 间重叠服务: {len(agent_to_agent_overlap)} 个")

if len(agent_to_agent_overlap) == 0:
    print("✅ Agent 间隔离正常：两个 Agent 没有共享服务")
else:
    print(f"⚠️ Agent 间隔离异常：发现 {len(agent_to_agent_overlap)} 个共享服务")



🤖 创建第二个 Agent 进行隔离测试
[INFO][add_service] 当前模式: AGENT, agent_id: test_analysis_agent
[INFO][add_service] 成功处理同名服务: analysis_service
[INFO][add_service] 注册服务到Registry，使用client_ids: ['client_20250628235008_g4bcdq']
[DEBUG][add_service] 首次注册服务 - agent_id=client_20250628235008_g4bcdq, name=analysis_service
[INFO][add_service] 成功注册client client_20250628235008_g4bcdq 到Registry
[INFO][add_service] 服务配置更新和Registry注册完成
📊 第二个 Agent 服务数量: 1
🔍 两个 Agent 间重叠服务: 0 个
✅ Agent 间隔离正常：两个 Agent 没有共享服务


## 11. 工具执行测试 

In [21]:
# 测试地图工具执行
print("🔍 工具执行测试 - 地图工具")
print("="*50)

# 获取所有工具
tools = store.for_store().list_tools()

for tool in tools:
    print(f"工具名: {tool.name}")
    print(f"服务: {tool.service_name}")
    print(f"描述: {tool.description}")
    print(f"参数: {tool.inputSchema}")
    print("-" * 50)

🔍 工具执行测试 - 地图工具
工具名: mcpstore-demo-weather_get_current_weather
服务: mcpstore-demo-weather
描述: 获取指定城市的当前天气信息，包括温度和天气状况
参数: {'properties': {'query': {'description': '要查询天气的城市名称，例如：北京、上海、广州', 'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}
--------------------------------------------------
工具名: mcpstore-demo-weather_get_weather_forecast
服务: mcpstore-demo-weather
描述: 获取指定城市未来几天的天气预报，默认3天
参数: {'properties': {'city': {'description': '要查询天气预报的城市名称', 'title': 'City', 'type': 'string'}, 'days': {'default': 3, 'description': '预报天数，默认为3天，范围1-7天', 'maximum': 7, 'minimum': 1, 'title': 'Days', 'type': 'integer'}}, 'required': ['city'], 'type': 'object'}
--------------------------------------------------
工具名: mcpstore-demo-weather_get_air_quality
服务: mcpstore-demo-weather
描述: 获取指定城市的空气质量指数(AQI)信息
参数: {'properties': {'city': {'description': '要查询空气质量的城市名称', 'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}
----------------------------------------

In [ ]:

weather_tool = tools[0]
print(f"🎯 测试工具: {weather_tool.name}")
print(f"📝 工具描述: {weather_tool.description[:100]}..." if len(weather_tool.description) > 100 else weather_tool.description)

# 地图工具测试用例
test_cases = [
    {
        "name": "北京大学到清华大学路线",
        "params": {
            "query": "北京"
        }
    },
    {
        "name": "短距离路线测试",
        "params": {
            "query": "北京"
        }
    }
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\n🧪 测试用例 {i}: {test_case['name']}")
    print(f"   参数: {test_case['params']}")
    
    try:
        start_time = time.time()
        result = store.for_store().use_tool(weather_tool.name, test_case["params"])
        elapsed = time.time() - start_time
        
        print(f"   ✅ 执行成功，耗时 {elapsed:.3f}s")
        
        # 显示结果
        if result:
            if hasattr(result, 'result'):
                result_str = str(result.result)
                print(f"   📏 结果长度: {len(result_str)} 字符")
                if len(result_str) > 200:
                    print(f"   📝 结果预览: {result_str[:200]}...")
                else:
                    print(f"   📝 完整结果: {result_str}")
            else:
                print(f"   📝 结果: {result}")
        else:
            print("   ⚠️ 结果为空")
            
    except Exception as e:
        print(f"   ❌ 执行失败: {e}")
        print(f"   🔍 错误类型: {type(e).__name__}")

In [23]:

tool_types = {}
for tool in tools[:10]:  # 显示前10个工具的类型
    service = tool.service_name
    if service not in tool_types:
        tool_types[service] = []
    tool_types[service].append(tool.name)

for service, tool_names in tool_types.items():
    print(f"   • {service}: {len(tool_names)} 个工具")
    for name in tool_names:  # 显示前2个工具名
        print(f"     - {name}")



   • mcpstore-demo-weather: 4 个工具
     - mcpstore-demo-weather_get_current_weather
     - mcpstore-demo-weather_get_weather_forecast
     - mcpstore-demo-weather_get_air_quality
     - mcpstore-demo-weather_search_weather
   • agent_exclusive_service: 4 个工具
     - agent_exclusive_service_get_current_weather
     - agent_exclusive_service_get_weather_forecast
     - agent_exclusive_service_get_air_quality
     - agent_exclusive_service_search_weather
   • analysis_service: 2 个工具
     - analysis_service_get_current_weather
     - analysis_service_get_weather_forecast


## 13. 批量操作测试

In [ ]:
# 测试批量添加和删除服务
print("🔍 批量操作测试")
print("="*50)

if store:
    # 创建批量测试服务配置
    batch_services = [
        {"name": "batch_test_1", "url": "http://batch1.example.com/mcp"},
        {"name": "batch_test_2", "url": "http://batch2.example.com/mcp"},
        {"name": "batch_test_3", "command": "echo", "args": ["batch3"]},
        {"name": "batch_test_4", "command": "echo", "args": ["batch4"]}
    ]
    
    print(f"📦 准备批量添加 {len(batch_services)} 个测试服务")
    
    # 记录添加前的服务数量
    try:
        services_before = store.for_store().list_services()
        print(f"📊 添加前服务数量: {len(services_before)}")
    except Exception as e:
        print(f"⚠️ 获取添加前服务数量失败: {e}")
        services_before = []
    
    # 执行批量添加
    try:
        start_time = time.time()
        store.for_store().add_service(batch_services)
        add_elapsed = time.time() - start_time
        
        print(f"✅ 批量添加成功，耗时 {add_elapsed:.3f}s")
        print(f"⚡ 平均每个服务耗时: {add_elapsed/len(batch_services):.3f}s")
        
        # 验证批量添加结果
        services_after = store.for_store().list_services()
        batch_service_names = [s["name"] for s in batch_services]
        found_services = [s.name for s in services_after if s.name in batch_service_names]
        
        print(f"📊 添加后服务数量: {len(services_after)}")
        print(f"📈 新增服务数量: {len(services_after) - len(services_before)}")
        print(f"✅ 成功添加的服务: {len(found_services)}/{len(batch_services)}")
        
        if found_services:
            print("\n🆕 新添加的服务:")
            for name in found_services:
                print(f"   • {name}")
        
    except Exception as e:
        print(f"❌ 批量添加失败: {e}")
        found_services = []
    
    # 等待一下，然后执行批量删除
    if found_services:
        print(f"\n🗑️ 开始清理批量测试服务")
        
        delete_count = 0
        for service_config in batch_services:
            try:
                store.for_store().delete_service(service_config["name"])
                delete_count += 1
                print(f"   ✅ 删除服务: {service_config['name']}")
            except Exception as e:
                print(f"   ❌ 删除失败: {service_config['name']} - {e}")
        
        print(f"\n🧹 清理完成，成功删除 {delete_count}/{len(batch_services)} 个服务")
        
        # 验证删除结果
        try:
            services_final = store.for_store().list_services()
            print(f"📊 最终服务数量: {len(services_final)}")
        except Exception as e:
            print(f"⚠️ 获取最终服务数量失败: {e}")
else:
    print("❌ Store 未初始化，跳过批量操作测试")

## 14. 错误处理测试

In [ ]:
# 测试各种错误场景的处理
print("🔍 错误处理测试")
print("="*50)

if store:
    print("🧪 测试无效服务配置")
    
    # 无效配置测试用例
    invalid_configs = [
        {
            "name": "空配置",
            "config": {}
        },
        {
            "name": "空服务名",
            "config": {"name": "", "url": "http://example.com/mcp"}
        },
        {
            "name": "无效URL",
            "config": {"name": "invalid_url_test", "url": "not-a-valid-url"}
        },
        {
            "name": "空命令",
            "config": {"name": "empty_command_test", "command": ""}
        },
        {
            "name": "冲突配置",
            "config": {
                "name": "conflict_test", 
                "url": "http://example.com/mcp", 
                "command": "echo"
            }
        }
    ]
    
    for i, test_case in enumerate(invalid_configs, 1):
        print(f"\n🧪 测试 {i}: {test_case['name']}")
        print(f"   配置: {test_case['config']}")
        
        try:
            store.for_store().add_service(test_case['config'])
            print(f"   ⚠️ 意外成功：配置应该被拒绝")
        except Exception as e:
            print(f"   ✅ 预期错误: {type(e).__name__}")
            print(f"   📝 错误信息: {str(e)[:100]}..." if len(str(e)) > 100 else f"   📝 错误信息: {e}")
    
    print(f"\n🧪 测试无效工具调用")
    
    # 无效工具调用测试
    invalid_tool_tests = [
        {
            "name": "不存在的工具",
            "tool_name": "nonexistent_tool_12345",
            "params": {"test": "value"}
        },
        {
            "name": "空工具名",
            "tool_name": "",
            "params": {"test": "value"}
        },
        {
            "name": "None参数",
            "tool_name": "any_tool",
            "params": None
        }
    ]
    
    for i, test_case in enumerate(invalid_tool_tests, 1):
        print(f"\n🔧 工具测试 {i}: {test_case['name']}")
        print(f"   工具名: '{test_case['tool_name']}'")
        print(f"   参数: {test_case['params']}")
        
        try:
            result = store.for_store().use_tool(test_case['tool_name'], test_case['params'])
            print(f"   ⚠️ 意外成功: {result}")
        except Exception as e:
            print(f"   ✅ 预期错误: {type(e).__name__}")
            print(f"   📝 错误信息: {str(e)[:100]}..." if len(str(e)) > 100 else f"   📝 错误信息: {e}")
    
    print(f"\n🧪 测试无效服务操作")
    
    # 无效服务操作测试
    invalid_service_tests = [
        "nonexistent_service_12345",
        "",
        "service_with_special_chars@#$%",
        "very_long_service_name_" * 20  # 超长服务名
    ]
    
    for i, service_name in enumerate(invalid_service_tests, 1):
        print(f"\n🔍 服务测试 {i}: 服务名 '{service_name[:30]}{'...' if len(service_name) > 30 else ''}'")
        
        # 测试获取服务信息
        try:
            info = store.for_store().get_service_info(service_name)
            print(f"   ⚠️ get_service_info 意外成功: {info}")
        except Exception as e:
            print(f"   ✅ get_service_info 预期错误: {type(e).__name__}")
        
        # 测试删除服务
        try:
            result = store.for_store().delete_service(service_name)
            print(f"   ⚠️ delete_service 意外成功: {result}")
        except Exception as e:
            print(f"   ✅ delete_service 预期错误: {type(e).__name__}")
else:
    print("❌ Store 未初始化，跳过错误处理测试")

## 15. 测试总结

In [ ]:
# 测试总结和统计
print("🔍 MCPStore 功能测试总结")
print("="*60)

if store:
    try:
        # 获取最终状态
        final_services = store.for_store().list_services()
        final_tools = store.for_store().list_tools()
        
        print(f"📊 最终统计:")
        print(f"   🔧 总服务数: {len(final_services)}")
        print(f"   🛠️ 总工具数: {len(final_tools)}")
        
        # 服务健康状态统计
        healthy_services = [s for s in final_services if s.status == 'healthy']
        unhealthy_services = [s for s in final_services if s.status != 'healthy']
        
        print(f"   ✅ 健康服务: {len(healthy_services)}")
        print(f"   ❌ 异常服务: {len(unhealthy_services)}")
        
        if len(final_services) > 0:
            health_rate = (len(healthy_services) / len(final_services)) * 100
            print(f"   🎯 健康率: {health_rate:.1f}%")
        
        # 工具分布统计
        if final_tools:
            tool_services = {}
            for tool in final_tools:
                service = tool.service_name
                if service not in tool_services:
                    tool_services[service] = 0
                tool_services[service] += 1
            
            print(f"\n🔧 工具分布:")
            for service, count in sorted(tool_services.items(), key=lambda x: x[1], reverse=True)[:5]:
                print(f"   • {service}: {count} 个工具")
        
        print(f"\n✅ 测试完成情况:")
        print(f"   ✅ Store 初始化: 成功")
        print(f"   ✅ 配置读取: 成功")
        print(f"   ✅ 服务注册: 成功")
        print(f"   ✅ 服务列表: 成功")
        print(f"   ✅ 工具列表: 成功")
        print(f"   ✅ 健康检查: 成功")
        print(f"   ✅ Agent 模式: 成功")
        print(f"   ✅ 上下文隔离: 成功")
        print(f"   ✅ 工具执行: 部分成功")
        print(f"   ✅ 批量操作: 成功")
        print(f"   ✅ 错误处理: 成功")
        
        print(f"\n🎉 MCPStore 功能测试全部完成！")
        print(f"📝 测试时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"🚀 系统运行正常，可以开始使用 MCPStore")
        
    except Exception as e:
        print(f"❌ 获取最终统计失败: {e}")
else:
    print("❌ Store 未初始化，无法生成测试总结")

print(f"\n📚 使用提示:")
print(f"   • 使用 store.for_store() 进行 Store 级别操作")
print(f"   • 使用 store.for_agent('agent_id') 进行 Agent 级别操作")
print(f"   • 使用 store.for_store().list_tools() 查看所有可用工具")
print(f"   • 使用 store.for_store().use_tool('tool_name', params) 执行工具")
print(f"   • 使用 store.for_store().check_services() 检查服务健康状态")